# Analyze NBA Matchup

## Module Imports

In [1]:
from bs4 import BeautifulSoup as Soup
from bs4 import Comment
from sys import exit
from os import path
import requests
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from datetime import date

## Global Variables and Settings

In [2]:
# Display all columns of DataFrames
pd.options.display.max_columns = None

# Print all output in a cell not just the last piece of output
InteractiveShell.ast_node_interactivity = "all"

# Location of sports data
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

# Lists of columns relating to percents, spread percents, and total percents
pct_cols = ['Cover Pct', 'Over Pct', '1H Cover Pct', '1H Over Pct', '2H Cover Pct', '2H Over Pct', '1Q Cover Pct', '1Q Over Pct', '2Q Cover Pct', '2Q Over Pct', '3Q Cover Pct', '3Q Over Pct', '4Q Cover Pct', '4Q Over Pct']

spread_cols = ['Cover Pct', '1H Cover Pct', '2H Cover Pct', '1Q Cover Pct', '2Q Cover Pct', '3Q Cover Pct', '4Q Cover Pct']

total_cols = ['Over Pct', '1H Over Pct', '2H Over Pct', '1Q Over Pct', '2Q Over Pct', '3Q Over Pct', '4Q Over Pct']

## Load Data

In [3]:
def load_data():
    # Load data from csv and reset the index
    game_data = pd.read_csv(path.join(DATA_DIR,'scraped_nba_combined_2021_compressed.csv'))
    game_data = game_data.set_index('Index')

    # Convert possible rows to numeric values
    for col in game_data.columns:
        try:
            game_data[col] = game_data.to_numeric(df[col])
        except:
            continue

    return game_data

In [4]:
game_data = load_data()

latest_date = str(game_data.loc[game_data.index[len(game_data.index) - 1], 'Date'])

year = int(latest_date[0:4])
month = int(latest_date[4:6])
day = int(latest_date[6:8])

toall = date.today()
toyear = int(toall.strftime("%Y"))
tomonth = int(toall.strftime("%m"))
today = int(toall.strftime("%d"))

if toyear >= year and (tomonth > month or (tomonth == month and today > day + 1)):
    os.system('python update_nba_boxscores_compressed.py')
    os.system('python update_nba_odds_compressed.py')
    os.system('python combine_boxscore_odds_compressed.py')

    game_data = load_data()

0

0

0

## Helper Functions

In [5]:
def get_url_from_team(team, year, games = '', prefix = 'https://www.basketball-reference.com/teams'):
    return f'{prefix}/{team}/{year}{games}.html'

In [6]:
def get_soup(team, year, games = ''):
    url = get_url_from_team(team, year, games)
    print(url)
    response = requests.get(url)
    if not 200 <= response.status_code < 300:
        exit('Invalid Team')
    return Soup(response.content, 'html.parser')

In [7]:
def parse_row(row):
    result = [ x.string for x in row.find_all('td') ]
    return result

In [8]:
def table_to_df(table, overheader = 0):
    cols = table.find('thead').find_all('tr')[overheader].find_all('th')
    cols = [ col.string for col in cols ]
    cols
    
    stat_table = table.find('tbody')
    stat_table
        
    rows = stat_table.find_all('tr')
    rows
    
    headers = [ row.find('th').string for row in rows ]
    headers = [ header for header in headers if header != 'G' ]
    
    list_of_parsed_rows = [ parse_row(row) for row in rows[0:len(rows)] ]
    list_of_parsed_rows = [ row for row in list_of_parsed_rows if row != [] ]
    list_of_parsed_rows
    
    df = pd.DataFrame(list_of_parsed_rows)
    df.insert(0, '', headers)
    df.columns = cols
    
    return df

In [9]:
def convert_df_to_int(df):
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            continue
    return df

In [10]:
def dfs_to_composite_df(team, dfs):
    df_basic = dfs[0].iloc[[1], :]
    df_basic.iloc[0, 0] = f'{team}/G'
    df_basic = df_basic.set_index(df_basic.columns[0])
    df_basic_lg = dfs[0].iloc[[2], :]
    df_basic_lg.iloc[0, 0] = f'{team}/LG'
    df_basic_lg = df_basic_lg.set_index(df_basic_lg.columns[0])
    df_advanced = dfs[1].iloc[[0], :]
    df_advanced.iloc[0, 0] = f'{team}/G'
    df_advanced = df_advanced.set_index(df_advanced.columns[0])
    df_advanced_lg = dfs[1].iloc[[1], :]
    df_advanced_lg.iloc[0, 0] = f'{team}/LG'
    df_advanced_lg = df_advanced_lg.set_index(df_advanced_lg.columns[0])
    row_1 = pd.concat([df_basic, df_advanced], axis = 1)
    row_2 = pd.concat([df_basic_lg, df_advanced_lg], axis = 1)
    composite_df = row_1.append(row_2)
    composite_df.columns = list(composite_df.columns[0:39]) + [ f'OPP {col}' for col in composite_df.columns[39:] ]
    return composite_df

In [11]:
def uncompress_table(table):
    rows = []

    for i in range(len(table.index)):
        game = table.iloc[i, :].to_dict()
        row_1 = { (key.split('Home ')[1] if 'Home' in key else key):(val) for (key, val) in game.items() if 'Away' not in key }
        row_2 = { (key.split('Away ')[1] if 'Away' in key else key):(val) for (key, val) in game.items() if 'Home' not in key }
        rows = rows + [row_1, row_2]

    df = pd.DataFrame(rows)
    df['Date'] = df['Date'].astype(str)
    df['Index'] = df[['Date', 'Team']].agg('_'.join, axis = 1)
    df = df.drop(['Date', 'Team'], axis = 1)
    df = df.set_index('Index')
    return df


In [12]:
def split_game_df(df):
    boxscores = []

    i = 0
    while i < len(df.index):
        boxscore = df.iloc[i: i + 2, :]
        new_index = [boxscore.index[0].split('_')[1], boxscore.index[1].split('_')[1]]
        date = boxscore.index[0].split('_')[0]
        boxscore.index = new_index
        boxscore.index.name = date
        boxscores.append(boxscore)
        i += 2

    return boxscores

## Team Statistics

In [13]:
def get_team_misc(soup, team, year):    
    div = soup.find('div', {'id': 'all_team_misc'})
    comment = div.find(string = lambda text: isinstance(text, Comment))
    table = Soup(comment, 'html.parser')
    table

    return table_to_df(table, 1)

In [14]:
def get_team_stats(soup, team, year, prompt = 'Team to research: '):    
    div = soup.find('div', {'id': 'all_team_and_opponent'})
    comment = div.find(string = lambda text: isinstance(text, Comment))
    table = Soup(comment, 'html.parser')
    
    return table_to_df(table, 0)

In [15]:
def get_team_dfs(team, year = 2021):
    soup = get_soup(team, year)
    
    main_df = get_team_stats(soup, team, year)
    
    misc_df = get_team_misc(soup, team, year)
    
    return [main_df, misc_df]

## Game Statistics

In [16]:
def get_game_stats(dateteam):
    # Construct boxscore URL from date and home team
    components = dateteam.split('_')
    date = components[0]
    home = components[1]

    url = f"https://www.basketball-reference.com/boxscores/{date}0{home}.html"


    # Get HTML soup for boxscore 
    response = requests.get(url)
    if not 200 <= response.status_code < 300:
        return None
    soup = Soup(response.content, 'html.parser')
    

    # Get line table
    line_div = soup.find('div', {'id': 'all_line_score'})
    line_comment = line_div.find(string = lambda text: isinstance(text, Comment))
    line_table = Soup(line_comment, 'html.parser')
    

    # Get four key factors table
    factors_div = soup.find('div', {'id': 'all_four_factors'})
    factors_comment = factors_div.find(string = lambda text: isinstance(text, Comment))
    factors_table = Soup(factors_comment, 'html.parser')
    

    # Convert tables to DataFrames and combine them
    line_df = table_to_df(line_table, 1)
    line_df = line_df.set_index(line_df.columns[0])
    factors_df = table_to_df(factors_table, 1)
    factors_df = factors_df.set_index(factors_df.columns[0])
    
    df = pd.concat([line_df, factors_df], axis = 1)
    
    
    return df

In [17]:
def get_game_url(dateteam):
    components = dateteam.split('_')
    date = components[0]
    home = components[1]

    url = f"https://www.basketball-reference.com/boxscores/{date}0{home}.html"


## Team Trends

In [18]:
def get_team_trends(team):
    home = game_data.loc[game_data['Home Team'] == team, :]

    home_cols = [ col.split('Home ')[1] if 'Home' in col else f"Opp {col.split('Away ')[1]}" if 'Away' in col else col for col in home.columns ]
    home.columns = home_cols

    away = game_data.loc[game_data['Away Team'] == team, :]
    away_cols = [ col.split('Away ')[1] if 'Away' in col else f"Opp {col.split('Home ')[1]}" if 'Home' in col else col for col in away.columns ]
    away.columns = away_cols

    return pd.concat([home, away])

In [19]:
def get_odds_data(team, recent = 0):
    combined = get_team_trends(team)
    
    if recent > 0 and recent <= len(combined.index):
        combined = combined.tail(recent)

    team_odds = {}
    team_odds['Team'] = team 
    team_odds['Games'] = len(combined.index)
    team_odds['Wins'] = len(combined.loc[combined['T'] > combined['Opp T']])
    team_odds['Losses'] = len(combined.loc[combined['T'] < combined['Opp T']])
    team_odds['Average Spread'] = combined['Spread'].mean()
    team_odds['Median Margin'] = (combined['T'] - combined['Opp T']).median()
    team_odds['Covers'] = len(combined.loc[combined['T'] - combined['Opp T'] > -combined['Spread']].index)
    team_odds['Not Covers'] = len(combined.loc[combined['T'] - combined['Opp T'] < -combined['Spread']].index)
    team_odds['Pushes'] = len(combined.loc[combined['T'] - combined['Opp T'] == -combined['Spread']].index)
    team_odds['Cover Pct'] = team_odds['Covers'] / (team_odds['Covers'] + team_odds['Not Covers'] + team_odds['Pushes'])
    team_odds['Average Total'] = combined['Over'].mean()
    team_odds['Median Points'] = (combined['T'] + combined['Opp T']).median()
    team_odds['Overs'] = len(combined.loc[combined['T'] + combined['Opp T'] > combined['Over']].index)
    team_odds['Unders'] = len(combined.loc[combined['T'] + combined['Opp T'] < combined['Over']].index)
    team_odds['Total Pushes'] = len(combined.loc[combined['T'] + combined['Opp T'] == combined['Over']].index)
    team_odds['Over Pct'] = team_odds['Overs'] / (team_odds['Overs'] + team_odds['Unders'] + team_odds['Total Pushes'])
    team_odds['1H Average Spread'] = combined['1H Spread'].mean()
    team_odds['1H Median Margin'] = (combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2']).median()
    team_odds['1H Covers'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] > -combined['1H Spread']].index)
    team_odds['1H Not Covers'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] < -combined['1H Spread']].index)
    team_odds['1H Pushes'] = len(combined.loc[combined['1'] + combined['2'] - combined['Opp 1'] - combined['Opp 2'] == -combined['1H Spread']].index)
    team_odds['1H Cover Pct'] = team_odds['1H Covers'] / (team_odds['1H Covers'] + team_odds['1H Not Covers'] + team_odds['1H Pushes'])    
    team_odds['1H Average Total'] = combined['1H Over'].mean()
    team_odds['1H Median Points'] = (combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2']).median()
    team_odds['1H Overs'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] > combined['1H Over']].index)
    team_odds['1H Unders'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] < combined['1H Over']].index)
    team_odds['1H Total Pushes'] = len(combined.loc[combined['1'] + combined['2'] + combined['Opp 1'] + combined['Opp 2'] == combined['1H Over']].index)
    team_odds['1H Over Pct'] = team_odds['1H Overs'] / (team_odds['1H Overs'] + team_odds['1H Unders'] + team_odds['1H Total Pushes'])
    team_odds['2H Average Spread'] = combined['2H Spread'].mean()
    team_odds['2H Median Margin'] = (combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4']).median()
    team_odds['2H Covers'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] > -combined['2H Spread']].index)
    team_odds['2H Not Covers'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] < -combined['2H Spread']].index)
    team_odds['2H Pushes'] = len(combined.loc[combined['3'] + combined['4'] - combined['Opp 3'] - combined['Opp 4'] == -combined['2H Spread']].index)
    team_odds['2H Cover Pct'] = team_odds['2H Covers'] / (team_odds['2H Covers'] + team_odds['2H Not Covers'] + team_odds['2H Pushes'])
    team_odds['2H Average Total'] = combined['2H Over'].mean()
    team_odds['2H Median Points'] = (combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4']).median()
    team_odds['2H Overs'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] > combined['2H Over']].index)
    team_odds['2H Unders'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] < combined['2H Over']].index)
    team_odds['2H Total Pushes'] = len(combined.loc[combined['3'] + combined['4'] + combined['Opp 3'] + combined['Opp 4'] == combined['2H Over']].index)
    team_odds['2H Over Pct'] = team_odds['2H Overs'] / (team_odds['2H Overs'] + team_odds['2H Unders'] + team_odds['2H Total Pushes'])
    team_odds['1Q Average Spread'] = combined['1Q Spread'].mean()
    team_odds['1Q Median Margin'] = (combined['1'] - combined['Opp 1']).median()
    team_odds['1Q Covers'] = len(combined.loc[combined['1'] - combined['Opp 1'] > -combined['1Q Spread']].index)
    team_odds['1Q Not Covers'] = len(combined.loc[combined['1'] - combined['Opp 1'] < -combined['1Q Spread']].index)
    team_odds['1Q Pushes'] = len(combined.loc[combined['1'] - combined['Opp 1'] == -combined['1Q Spread']].index)
    team_odds['1Q Cover Pct'] = team_odds['1Q Covers'] / (team_odds['1Q Covers'] + team_odds['1Q Not Covers'] + team_odds['1Q Pushes'])
    team_odds['1Q Average Total'] = combined['1Q Over'].mean()
    team_odds['1Q Median Points'] = (combined['1'] + combined['Opp 1']).median()
    team_odds['1Q Overs'] = len(combined.loc[combined['1'] + combined['Opp 1'] > combined['1Q Over']].index)
    team_odds['1Q Unders'] = len(combined.loc[combined['1'] + combined['Opp 1'] < combined['1Q Over']].index)
    team_odds['1Q Total Pushes'] = len(combined.loc[combined['1'] + combined['Opp 1'] == combined['1Q Over']].index)
    team_odds['1Q Over Pct'] = team_odds['1Q Overs'] / (team_odds['1Q Overs'] + team_odds['1Q Unders'] + team_odds['1Q Total Pushes'])
    team_odds['2Q Average Spread'] = combined['2Q Spread'].mean()
    team_odds['2Q Median Margin'] = (combined['2'] - combined['Opp 2']).median()
    team_odds['2Q Covers'] = len(combined.loc[combined['2'] - combined['Opp 2'] > -combined['2Q Spread']].index)
    team_odds['2Q Not Covers'] = len(combined.loc[combined['2'] - combined['Opp 2'] < -combined['2Q Spread']].index)
    team_odds['2Q Pushes'] = len(combined.loc[combined['2'] - combined['Opp 2'] == -combined['2Q Spread']].index)
    team_odds['2Q Cover Pct'] = team_odds['2Q Covers'] / (team_odds['2Q Covers'] + team_odds['2Q Not Covers'] + team_odds['2Q Pushes'])
    team_odds['2Q Average Total'] = combined['2Q Over'].mean()
    team_odds['2Q Median Points'] = (combined['2'] + combined['Opp 2']).median()
    team_odds['2Q Overs'] = len(combined.loc[combined['2'] + combined['Opp 2'] > combined['2Q Over']].index)
    team_odds['2Q Unders'] = len(combined.loc[combined['2'] + combined['Opp 2'] < combined['2Q Over']].index)
    team_odds['2Q Total Pushes'] = len(combined.loc[combined['2'] + combined['Opp 2'] == combined['2Q Over']].index)
    team_odds['2Q Over Pct'] = team_odds['2Q Overs'] / (team_odds['2Q Overs'] + team_odds['2Q Unders'] + team_odds['2Q Total Pushes'])
    team_odds['3Q Average Spread'] = combined['3Q Spread'].mean()
    team_odds['3Q Median Margin'] = (combined['3'] - combined['Opp 3']).median()
    team_odds['3Q Covers'] = len(combined.loc[combined['3'] - combined['Opp 3'] > -combined['3Q Spread']].index)
    team_odds['3Q Not Covers'] = len(combined.loc[combined['3'] - combined['Opp 3'] < -combined['3Q Spread']].index)
    team_odds['3Q Pushes'] = len(combined.loc[combined['3'] - combined['Opp 3'] == -combined['3Q Spread']].index)
    team_odds['3Q Cover Pct'] = team_odds['3Q Covers'] / (team_odds['3Q Covers'] + team_odds['3Q Not Covers'] + team_odds['3Q Pushes'])
    team_odds['3Q Average Total'] = combined['3Q Over'].mean()
    team_odds['3Q Median Points'] = (combined['3'] + combined['Opp 3']).median()
    team_odds['3Q Overs'] = len(combined.loc[combined['3'] + combined['Opp 3'] > combined['3Q Over']].index)
    team_odds['3Q Unders'] = len(combined.loc[combined['3'] + combined['Opp 3'] < combined['3Q Over']].index)
    team_odds['3Q Total Pushes'] = len(combined.loc[combined['3'] + combined['Opp 3'] == combined['3Q Over']].index)
    team_odds['3Q Over Pct'] = team_odds['3Q Overs'] / (team_odds['3Q Overs'] + team_odds['3Q Unders'] + team_odds['3Q Total Pushes'])
    team_odds['4Q Average Spread'] = combined['4Q Spread'].mean()
    team_odds['4Q Median Margin'] = (combined['4'] - combined['Opp 4']).median()
    team_odds['4Q Covers'] = len(combined.loc[combined['4'] - combined['Opp 4'] > -combined['4Q Spread']].index)
    team_odds['4Q Not Covers'] = len(combined.loc[combined['4'] - combined['Opp 4'] < -combined['4Q Spread']].index)
    team_odds['4Q Pushes'] = len(combined.loc[combined['4'] - combined['Opp 4'] == -combined['4Q Spread']].index)
    team_odds['4Q Cover Pct'] = team_odds['4Q Covers'] / (team_odds['4Q Covers'] + team_odds['4Q Not Covers'] + team_odds['4Q Pushes'])
    team_odds['4Q Average Total'] = combined['4Q Over'].mean()
    team_odds['4Q Median Points'] = (combined['4'] + combined['Opp 4']).median()
    team_odds['4Q Overs'] = len(combined.loc[combined['4'] + combined['Opp 4'] > combined['4Q Over']].index)
    team_odds['4Q Unders'] = len(combined.loc[combined['4'] + combined['Opp 4'] < combined['4Q Over']].index)
    team_odds['4Q Total Pushes'] = len(combined.loc[combined['4'] + combined['Opp 4'] == combined['4Q Over']].index)
    team_odds['4Q Over Pct'] = team_odds['4Q Overs'] / (team_odds['4Q Overs'] + team_odds['4Q Unders'] + team_odds['4Q Total Pushes'])

    pd.DataFrame(team_odds, index = [0])

    return team_odds

In [20]:
def compare_teams(team_1, team_2, recent = 0):
    matchup_rows = []

    matchup_rows.append(get_odds_data(team_1, recent))
    matchup_rows.append(get_odds_data(team_2, recent))

    return pd.DataFrame(matchup_rows)

## Analyze Matchup

In [21]:
# Get teams to analyze
team_1 = input('Team 1: ')
team_2 = input('Team 2: ')

In [22]:
# Get team 1 statistics DataFrames
team_1_dfs = get_team_dfs(team_1)
team_1_dfs[0]
team_1_dfs[1]

https://www.basketball-reference.com/teams/PHO/2021.html


,,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,72,17480,3118,6357,.490,940,2490,.378,2178,3867,.563,1124,1347,.834,630,2462,3092,1939,517,312,902,1374,8300
1,Team/G,None,242.8,43.3,88.3,.490,13.1,34.6,.378,30.3,53.7,.563,15.6,18.7,.834,8.8,34.2,42.9,26.9,7.2,4.3,12.5,19.1,115.3
2,Lg Rank,None,1,2,16,2,13,15,7,9,16,2,28,29,2,28,18,23,3,19,25,4,14,7
3,Year/Year,None,0.7%,5.2%,0.3%,+.023,14.7%,8.8%,+.019,1.5%,-4.6%,+.034,-21.5%,-21.5%,+.001,-10.3%,1.3%,-1.3%,-1.1%,-6.2%,9.1%,-15.1%,-13.4%,1.5%
4,Opponent,72,17480,2916,6248,.467,836,2359,.354,2080,3889,.535,1213,1560,.778,674,2401,3075,1652,498,258,981,1294,7881
5,Opponent/G,None,242.8,40.5,86.8,.467,11.6,32.8,.354,28.9,54.0,.535,16.8,21.7,.778,9.4,33.3,42.7,22.9,6.9,3.6,13.6,18.0,109.5
6,Lg Rank,None,1,11,9,15,7,10,5,18,15,17,12,13,15,6,10,9,4,5,1,17,28,7
7,Year/Year,None,0.7%,-1.8%,-0.7%,-.005,-0.0%,2.4%,-.009,-2.5%,-2.5%,-.000,-12.7%,-11.5%,-.010,2.9%,-2.5%,-1.4%,-2.8%,-12.3%,-34.8%,-12.7%,-20.6%,-3.5%


,,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attendance
0,Team,51,21,49,23,5.82,-0.15,5.67,117.2,111.3,97.2,.212,.392,.564,11.5,20.8,.177,.534,12.4,78.5,.194,Phoenix Suns Arena,"104,027"
1,Lg Rank,2,29,3,3,4,23,3,5,9,26,29,15,3,4,25,27,10,16,9,14,None,4


In [23]:
# Get team 2 statistics DataFrames
team_2_dfs = get_team_dfs(team_2)
team_2_dfs[0]
team_2_dfs[1]

https://www.basketball-reference.com/teams/MIL/2021.html


,,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,72,17330,3221,6610,.487,1038,2669,.389,2183,3941,.554,1169,1539,.760,741,2724,3465,1834,585,334,995,1244,8649
1,Team/G,None,240.7,44.7,91.8,.487,14.4,37.1,.389,30.3,54.7,.554,16.2,21.4,.760,10.3,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1
2,Lg Rank,None,22,1,1,3,5,8,5,8,13,4,23,16,23,12,1,2,14,7,19,15,2,1
3,Year/Year,None,-0.1%,3.3%,1.0%,+.011,4.5%,-4.7%,+.034,2.8%,5.2%,-.013,-11.3%,-13.3%,+.017,8.7%,-10.4%,-6.9%,-1.6%,12.8%,-21.1%,-8.5%,-11.9%,1.2%
4,Opponent,72,17330,3052,6688,.456,1068,2778,.384,1984,3910,.507,1053,1350,.780,693,2435,3128,1804,507,348,947,1320,8225
5,Opponent/G,None,240.7,42.4,92.9,.456,14.8,38.6,.384,27.6,54.3,.507,14.6,18.8,.780,9.6,33.8,43.4,25.1,7.0,4.8,13.2,18.3,114.2
6,Lg Rank,None,22,23,30,5,30,28,29,10,17,2,3,1,17,9,13,13,17,6,15,20,24,22
7,Year/Year,None,-0.1%,8.8%,-1.2%,+.042,6.2%,-1.9%,+.029,10.3%,-0.7%,+.051,-12.6%,-11.8%,-.007,1.4%,-6.9%,-5.2%,3.7%,-4.5%,8.2%,-6.8%,-15.6%,5.2%


,,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attendance
0,Team,46,26,48,24,5.89,-0.32,5.57,117.2,111.4,102.2,.233,.404,.566,12.0,23.3,.177,.536,11.5,79.7,.157,Fiserv Forum,"64,780"
1,Lg Rank,7,24,5,5,3,30,4,6,10,2,24,12,2,12,9,26,13,24,3,1,None,10


In [24]:
# Create and combine composite main and miscellaneous DataFranes
composite_df = dfs_to_composite_df(team_1, team_1_dfs).append(dfs_to_composite_df(team_2, team_2_dfs))

# Drop irrelevant columns
composite_df = composite_df.drop(labels = ['G', 'OPP Arena', 'OPP Attendance'], axis = 1)

# Convert relevant rows to numeric values
composite_df.iloc[[0, 2], :] = convert_df_to_int(composite_df.iloc[[0, 2], :])

# Reorder rows
composite_df = composite_df.reindex([composite_df.index[0], composite_df.index[2], composite_df.index[1], composite_df.index[3]])

# Calculate means of team stats and add it to the combined DataFrame
means = composite_df.iloc[[0, 1], :].mean()
means.name = 'Mean'
composite_df.append(means)

# Get boxscores of head-to-head games
head_to_heads = game_data.loc[((game_data['Home Team'] == team_1) & (game_data['Away Team'] == team_2)) | ((game_data['Home Team'] == team_2) & (game_data['Away Team'] == team_1))]
# head_to_heads

if len(head_to_heads) > 0:
    # Convert the head-to-head table to an uncompressed format
    uncompressed = uncompress_table(head_to_heads)
    # uncompressed

    # Print out the games as individual boxscores
    boxscores = split_game_df(uncompressed)
    for boxscore in boxscores:
        boxscore

C:\Users\Harry\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Harry\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
<ipython-input-9-3536e1e3a1fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,OPP eFG%,OPP TOV%,OPP DRB%,OPP FT/FGA
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PHO/G,242.8,43.3,88.3,0.49,13.1,34.6,0.378,30.3,53.7,0.563,15.6,18.7,0.834,8.8,34.2,42.9,26.9,7.2,4.3,12.5,19.1,115.3,51.0,21.0,49.0,23.0,5.82,-0.15,5.67,117.2,111.3,97.2,0.212,0.392,0.564,11.5,20.8,0.177,0.534,12.4,78.5,0.194
MIL/G,240.7,44.7,91.8,0.487,14.4,37.1,0.389,30.3,54.7,0.554,16.2,21.4,0.76,10.3,37.8,48.1,25.5,8.1,4.6,13.8,17.3,120.1,46.0,26.0,48.0,24.0,5.89,-0.32,5.57,117.2,111.4,102.2,0.233,0.404,0.566,12.0,23.3,0.177,0.536,11.5,79.7,0.157
PHO/LG,1,2,16,2,13,15,7,9,16,2,28,29,2,28,18,23,3,19,25,4,14,7,2,29,3,3,4,23,3,5,9,26,29,15,3,4,25,27,10,16,9,14
MIL/LG,22,1,1,3,5,8,5,8,13,4,23,16,23,12,1,2,14,7,19,15,2,1,7,24,5,5,3,30,4,6,10,2,24,12,2,12,9,26,13,24,3,1
Mean,241.75,44.0,90.05,0.4885,13.75,35.85,0.3835,30.3,54.2,0.5585,15.9,20.05,0.797,9.55,36.0,45.5,26.2,7.65,4.45,13.15,18.2,117.7,48.5,23.5,48.5,23.5,5.855,-0.235,5.62,117.2,111.35,99.7,0.2225,0.398,0.565,11.75,22.05,0.177,0.535,11.95,79.1,0.1755


,1,2,3,4,OT,2OT,3OT,T,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
20210210,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MIL,32,39,30,23,0,0,0,124,-4.0,-105,-170,228.5,-105,228.5,-105,-2.0,-110,-150,114.0,-110,114.0,-110,3.0,-135.0,130.0,115.5,105.0,115.5,-125.0,-1.0,-110.0,-135,56.5,-110.0,56.5,-110.0,-0.5,-115.0,-133.0,56.0,-110.0,56.0,-110.0,1.5,-125.0,120.0,58.0,100.0,58.0,-120.0,-0.5,-115.0,NaN,56.5,-110.0,56.5,-110.0
PHO,32,27,39,27,0,0,0,125,4.0,-105,160,228.5,-105,228.5,-105,2.0,-110,130,114.0,-110,114.0,-110,-3.0,115.0,-150.0,115.5,105.0,115.5,-125.0,1.0,-110.0,115,56.5,-110.0,56.5,-110.0,0.5,-105.0,108.0,56.0,-110.0,56.0,-110.0,-1.5,105.0,-140.0,58.0,100.0,58.0,-120.0,0.5,-105.0,NaN,56.5,-110.0,56.5,-110.0


,1,2,3,4,OT,2OT,3OT,T,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
20210419,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PHO,24,31,28,33,12,0,0,128,3.0,-105,131,233.0,-105,233.0,-105,1.0,-110,113,114.5,-110,114.5,-110,1.0,118.0,135.0,118.5,-103.0,118.5,-117.0,0.5,-110.0,103,57.5,-115.0,57.5,-105.0,0.5,-110.0,103.0,57.0,-110.0,57.0,-110.0,1.0,-110.0,120.0,58.5,-110.0,58.5,-110.0,0.5,-122.0,-110.0,58.5,-115.0,58.5,-105.0
MIL,25,29,36,26,11,0,0,127,-3.0,-105,-141,233.0,-105,233.0,-105,-1.0,-110,-133,114.5,-110,114.5,-110,-1.0,-138.0,-155.0,118.5,-103.0,118.5,-117.0,-0.5,-110.0,-123,57.5,-115.0,57.5,-105.0,-0.5,-110.0,-123.0,57.0,-110.0,57.0,-110.0,-1.0,-110.0,-140.0,58.5,-110.0,58.5,-110.0,-0.5,102.0,-110.0,58.5,-115.0,58.5,-105.0


,1,2,3,4,OT,2OT,3OT,T,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
20210706,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MIL,26,23,27,29,0,0,0,105,4.5,-105,185,219.5,-103,219.5,-107,2.5,100,160,108.0,-105,108.0,-105,2.0,-167.0,-120.0,110.0,-140.0,110.0,120.0,2.0,100.0,155,54.0,-105.0,54.0,-105.0,1.0,-110.0,115.0,53.5,-110.0,53.5,-110.0,1.0,-145.0,-115.0,57.0,133.0,57.0,-153.0,1.0,-150.0,-120.0,56.0,118.0,56.0,-138.0
PHO,30,27,35,26,0,0,0,118,-4.5,-105,-200,219.5,-103,219.5,-107,-2.5,-110,-170,108.0,-105,108.0,-105,-2.0,147.0,100.0,110.0,-140.0,110.0,120.0,-2.0,-110.0,-165,54.0,-105.0,54.0,-105.0,-1.0,-110.0,-135.0,53.5,-110.0,53.5,-110.0,-1.0,125.0,-105.0,57.0,133.0,57.0,-153.0,-1.0,130.0,100.0,56.0,118.0,56.0,-138.0


,1,2,3,4,OT,2OT,3OT,T,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
20210708,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MIL,29,16,33,30,0,0,0,108,4.5,-105,167,221.0,-105,221.0,-105,2.5,-109,147,108.5,-105,108.5,-105,-1.5,-108.0,-135.0,112.0,-105.0,112.0,-105.0,1.5,-105.0,135,54.0,-110.0,54.0,100.0,1.0,-115.0,110.0,53.5,-110.0,53.5,-110.0,-1.0,100.0,-130.0,55.5,-110.0,55.5,-110.0,-0.5,-115.0,-130.0,56.0,-110.0,56.0,-110.0
PHO,26,30,32,30,0,0,0,118,-4.5,-105,-178,221.0,-105,221.0,-105,-2.5,-101,-157,108.5,-105,108.5,-105,1.5,-102.0,125.0,112.0,-105.0,112.0,-105.0,-1.5,-105.0,-145,54.0,-110.0,54.0,100.0,-1.0,-105.0,-130.0,53.5,-110.0,53.5,-110.0,1.0,-120.0,110.0,55.5,-110.0,55.5,-110.0,0.5,-105.0,110.0,56.0,-110.0,56.0,-110.0


In [25]:
matchup_odds_df = compare_teams(team_1, team_2)
matchup_odds_df.index.name = 'All'
matchup_odds_df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .6 or v < .4) else "" for v in x], axis = 1, subset = pct_cols)

# matchup_odds_df.index.name = 'Spreads'
# matchup_odds_df.style.apply(lambda y: ["background: blue", "background: blue"] if (y[0] > .6 and y[1] < .4) or (y[0] < .4 and y[1] > .6) else ["", ""], axis = 0, subset = spread_cols)

# matchup_odds_df.index.name = 'Totals'
# matchup_odds_df.style.apply(lambda y: ["background: blue", "background: blue"] if (y[0] > .6 and y[1] > .6) or (y[0] < .4 and y[1] < .4) else ["", ""], axis = 0, subset = total_cols)

recent_odds_df = compare_teams(team_1, team_2, 6)
recent_odds_df.index.name = 'Recent'
recent_odds_df.style.apply(lambda x: ["background: green" if type(v) != str and (v > .7 or v < .3) else "" for v in x], axis = 1, subset = pct_cols)

,Team,Games,Wins,Losses,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
All,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,PHO,90,65,25,-4.205556,6.000000,55,33,2,0.611111,221.094444,222.000000,51,39,0,0.566667,-2.261111,4.000000,55,35,0,0.611111,110.061111,110.000000,45,42,3,0.500000,0.034091,1.500000,46,39,3,0.522727,111.954545,111.000000,45,42,1,0.511364,-1.283333,2.000000,53,36,1,0.588889,55.266667,56.000000,46,43,1,0.511111,-0.916667,2.000000,53,36,1,0.588889,54.672222,56.000000,49,41,0,0.544444,-0.633333,0.500000,44,42,4,0.488889,55.761111,57.000000,51,36,3,0.566667,-0.527778,0.000000,47,43,0,0.522222,54.933333,53.000000,37,51,2,0.411111
1,MIL,91,58,33,-6.148352,6.000000,42,49,0,0.461538,229.604396,230.000000,49,41,1,0.538462,-3.527473,5.000000,47,39,5,0.516484,114.598901,113.000000,43,46,2,0.472527,-0.438889,1.000000,45,44,1,0.500000,115.283333,115.000000,41,48,1,0.455556,-2.049451,3.000000,49,42,0,0.538462,57.450549,58.000000,48,41,2,0.527473,-1.357143,1.000000,44,45,2,0.483516,56.895604,58.000000,44,45,2,0.483516,-1.390110,1.000000,42,48,1,0.461538,57.802198,59.000000,51,35,5,0.560440,-0.994505,-1.000000,36,53,2,0.395604,56.901099,54.000000,35,56,0,0.384615


,Team,Games,Wins,Losses,Average Spread,Median Margin,Covers,Not Covers,Pushes,Cover Pct,Average Total,Median Points,Overs,Unders,Total Pushes,Over Pct,1H Average Spread,1H Median Margin,1H Covers,1H Not Covers,1H Pushes,1H Cover Pct,1H Average Total,1H Median Points,1H Overs,1H Unders,1H Total Pushes,1H Over Pct,2H Average Spread,2H Median Margin,2H Covers,2H Not Covers,2H Pushes,2H Cover Pct,2H Average Total,2H Median Points,2H Overs,2H Unders,2H Total Pushes,2H Over Pct,1Q Average Spread,1Q Median Margin,1Q Covers,1Q Not Covers,1Q Pushes,1Q Cover Pct,1Q Average Total,1Q Median Points,1Q Overs,1Q Unders,1Q Total Pushes,1Q Over Pct,2Q Average Spread,2Q Median Margin,2Q Covers,2Q Not Covers,2Q Pushes,2Q Cover Pct,2Q Average Total,2Q Median Points,2Q Overs,2Q Unders,2Q Total Pushes,2Q Over Pct,3Q Average Spread,3Q Median Margin,3Q Covers,3Q Not Covers,3Q Pushes,3Q Cover Pct,3Q Average Total,3Q Median Points,3Q Overs,3Q Unders,3Q Total Pushes,3Q Over Pct,4Q Average Spread,4Q Median Margin,4Q Covers,4Q Not Covers,4Q Pushes,4Q Cover Pct,4Q Average Total,4Q Median Points,4Q Overs,4Q Unders,4Q Total Pushes,4Q Over Pct
Recent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,PHO,6,5,1,-5.000000,8.000000,4,2,0,0.666667,220.083333,222.000000,4,2,0,0.666667,-2.583333,5.500000,4,2,0,0.666667,109.083333,103.500000,2,4,0,0.333333,-3.916667,1.500000,4,2,0,0.666667,114.250000,120.000000,4,2,0,0.666667,-1.666667,1.500000,3,3,0,0.500000,54.750000,51.000000,3,3,0,0.500000,-0.916667,3.500000,5,1,0,0.833333,54.250000,48.500000,1,5,0,0.166667,-1.583333,1.000000,3,2,1,0.500000,56.083333,60.500000,5,1,0,0.833333,-1.166667,-0.500000,3,3,0,0.500000,55.666667,57.500000,3,3,0,0.500000
1,MIL,6,5,1,-4.583333,11.000000,4,2,0,0.666667,224.583333,209.500000,2,4,0,0.333333,-3.250000,7.000000,4,1,1,0.666667,114.250000,110.000000,2,4,0,0.333333,0.083333,2.000000,4,2,0,0.666667,109.583333,98.000000,2,4,0,0.333333,-1.750000,6.500000,5,1,0,0.833333,57.000000,51.000000,2,4,0,0.333333,-0.916667,3.000000,4,2,0,0.666667,56.666667,60.000000,4,2,0,0.666667,-0.416667,2.000000,3,3,0,0.500000,55.583333,46.500000,1,5,0,0.166667,0.000000,-0.500000,1,4,1,0.166667,54.583333,51.500000,1,5,0,0.166667
